In [1]:
!pip install -q faiss-cpu sentence-transformers transformers

from google.colab import drive
drive.mount('/content/drive')

import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import textwrap



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.7 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
device = "cpu"
print("Using device:", device)

INDEX_PATH = "/content/drive/MyDrive/B5W6-Intelligent-Complaint-Analysis/vector_store/faiss_index.index"
META_PATH = "/content/drive/MyDrive/B5W6-Intelligent-Complaint-Analysis/vector_store/metadata.pkl"

# Load index & metadata
index = faiss.read_index(INDEX_PATH)
with open(META_PATH, "rb") as f:
    metadata = pickle.load(f)

# Sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Lightweight LLM
llm = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

def generate_response(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = llm.generate(input_ids, max_new_tokens=256)
    return tokenizer.decode(output[0], skip_special_tokens=True)


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [8]:
def generate_answer(question, top_k=5):
    # Embed the query
    query_emb = model.encode([question])[0].astype("float32")
    D, I = index.search(np.array([query_emb]), top_k)

    # Retrieve top chunks
    context_chunks = []
    retrieved_sources = []
    for i, idx in enumerate(I[0]):
        entry = metadata[idx]
        chunk_text = entry.get("text") or entry.get("chunk") or "N/A"
        context_chunks.append(chunk_text)
        retrieved_sources.append({
            "rank": i + 1,
            "product": entry.get("product", "N/A"),
            "text": chunk_text
        })

    # Create prompt
    context = "\n\n".join(context_chunks)
    prompt = f"""You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints. Use the following retrieved complaint excerpts to formulate your answer. If the context doesn't contain the answer, state that you don't have enough information.

Context:
{context}

Question: {question}
Answer:"""

    # Generate response
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer, retrieved_sources


In [12]:
query = "Why are refund delays so common in credit card issues?"
chunks = retrieve_similar_chunks(query, top_k=5)
prompt = format_prompt(chunks, query)
response = generate_response(prompt)

print("📌 Generated Answer:\n")
print(textwrap.fill(response, width=100))

print("\n📚 Top Sources:\n")
for chunk in chunks:
    print(f"🔹 Product: {chunk['product']}\n{textwrap.fill(chunk['text'], width=100)}\n")


📌 Generated Answer:

no justifiable reason that a credit or refund can not return money to an account as quickly as a
charge takes it there should be a

📚 Top Sources:

🔹 Product: Credit card
sometimes longer it seems that the delay has been normalized and institutionalized by the banking
industry and regulatory appears to not consider it important  there is no justifiable reason that a
credit or refund can not return money to an account as quickly as a charge takes it there should be
a

🔹 Product: Credit card
this is not a complaint about any particular credit card company although my recent experience is
with capital one it is exceedingly unfair that while credit card charges are applied to my account
instantaneously credits or refunds take several days sometimes longer it seems that the delay has
been

🔹 Product: Credit card
 i have never experienced such a problem with any other credit card this delay has caused
significant inconvenience  for instance i am unable to pay certain bil

In [13]:
import textwrap

# List of 5 representative questions
questions = [
    "Why are refund delays so common in credit card issues?",
    "How do customers describe problems with Buy Now, Pay Later services?",
    "Are there frequent complaints about unauthorized transactions in savings accounts?",
    "What are the most cited issues regarding virtual currency services?",
    "Do consumers often mention being charged twice without refund?"
]

# Run each question through the RAG pipeline
for i, query in enumerate(questions, start=1):
    print(f"\n🟩 Q{i}: {query}\n{'='*100}")

    try:
        chunks = retrieve_similar_chunks(query, top_k=5)
        prompt = format_prompt(chunks, query)
        response = generate_response(prompt)

        print("📌 Generated Answer:\n")
        print(textwrap.fill(response, width=100))

        print("\n📚 Top Sources:\n")
        for chunk in chunks[:2]:  # Limit to top 2 retrieved chunks
            print(f"🔹 Product: {chunk['product']}\n{textwrap.fill(chunk['text'], width=100)}\n")
    except Exception as e:
        print(f"❌ Error processing Q{i}: {str(e)}")



🟩 Q1: Why are refund delays so common in credit card issues?
📌 Generated Answer:

no justifiable reason that a credit or refund can not return money to an account as quickly as a
charge takes it there should be a

📚 Top Sources:

🔹 Product: Credit card
sometimes longer it seems that the delay has been normalized and institutionalized by the banking
industry and regulatory appears to not consider it important  there is no justifiable reason that a
credit or refund can not return money to an account as quickly as a charge takes it there should be
a

🔹 Product: Credit card
this is not a complaint about any particular credit card company although my recent experience is
with capital one it is exceedingly unfair that while credit card charges are applied to my account
instantaneously credits or refunds take several days sometimes longer it seems that the delay has
been


🟩 Q2: How do customers describe problems with Buy Now, Pay Later services?
📌 Generated Answer:

s lack of due diligence 